In [ ]:
from flask import Flask, render_template, request
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)

# Define functions get_number_of_pages, get_rating, get_book_recommendations as before

def get_number_of_pages(book_url):
    try:
        response = requests.get(book_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        pages_element = soup.find('dd', {'itemprop': 'numberOfPages', 'class': 'object'})

        if pages_element:
            return int(pages_element.get_text(strip=True).split()[0])  # Extract and convert to int

    except requests.exceptions.RequestException as e:
        print(f"Error fetching book page: {e}")

    return None

def get_rating(book_url):
    try:
        response = requests.get(book_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        rating_element = soup.find('span', {'itemprop': 'ratingValue'})

        if rating_element:
            return float(rating_element.get_text(strip=True))

    except requests.exceptions.RequestException as e:
        print(f"Error fetching book page: {e}")

    return None

def get_book_recommendations(category, max_pages=None, min_rating=None):
    base_url = "http://openlibrary.org/search.json"
    max_results = 5  # Number of recommendations

    # Construct the API request URL with additional parameters
    url = f"{base_url}?subject={category}&limit={max_results}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        docs = data.get('docs', [])

        # Extract information about the recommended books
        recommendations = []
        for doc in docs:
            title = doc.get('title', 'N/A')
            author = ', '.join(doc.get('author_name', ['N/A']))
            book_url = f"http://openlibrary.org{doc.get('key', 'N/A')}"
            
            # Use web scraping to get the number of pages and rating
            pages = get_number_of_pages(book_url)
            rating = get_rating(book_url)

            # Perform additional filtering based on page count and rating
            if (max_pages is None or (isinstance(pages, int) and pages <= max_pages)) and \
               (min_rating is None or (isinstance(rating, float) and rating >= min_rating)):
                recommendations.append({
                    'title': title,
                    'author': author,
                    'page_count': pages,
                    'rating': rating
                })

                if len(recommendations) == max_results:
                    break  # Stop once we reach the desired number of results

        return recommendations

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Open Library API: {e}")
        return None

# Get user input
category = input("Enter the category/genre you're interested in: ")
max_pages_str = input("Enter the maximum number of pages you prefer (or leave blank for no filter): ")
max_pages = int(max_pages_str) if max_pages_str.strip().isdigit() else None
min_rating_str = input("Enter the minimum rating you prefer (or leave blank for no filter): ")
min_rating = float(min_rating_str) if min_rating_str.strip().replace('.', '', 1).isdigit() else None

# Call the function
recommendations = get_book_recommendations(category, max_pages, min_rating)

# Print the recommendations
if recommendations:
    print(f"\nTop 5 book recommendations in the '{category}' category with "
          f"<= {max_pages} pages and >= {min_rating} rating:")
    for idx, book in enumerate(recommendations, start=1):
        print(f"{idx}. Title: {book['title']}, Author: {book['author']}, "
              f"Pages: {book['page_count']}, Rating: {book['rating']}")


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        category = request.form['category']
        max_pages = request.form['max_pages']
        min_rating = request.form['min_rating']

        recommendations = get_book_recommendations(category, max_pages, min_rating)

        if recommendations:
            return render_template('index.html', recommendations=recommendations)

    return render_template('index.html', recommendations=None)

if __name__ == '__main__':
    app.run(debug=False)
